XYZ-анализ — это метод управления запасами, используемый для классификации товаров или компонентов в зависимости от их стабильности или предсказуемости потребления или спроса. 

Фокусируется на предсказуемости спроса на товары.
X — товары с очень стабильным спросом.
Y — товары с умеренной стабильностью спроса.
Z — товары с нестабильным спросом.

In [29]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('sales_products_abc.csv')

In [4]:
# приводим столбец date к типу atetime с последущим округлением даты до месяца
df['date'] = pd.to_datetime(df['date']).dt.to_period('M')

In [24]:
# Рассчитываем суммарное количетво продашь товаров в разбивке по дате и типу товара
df_grupped = df.groupby(['date', 'product'], as_index=False).agg(sum_quantity = ('quantity', 'sum'))

In [39]:
#рассчитываеv коэффициент вариации, который представляет собой отношение стандартного отклонения к среднему значению.
df_cv = df_grupped.groupby('product', as_index=False)['sum_quantity'].apply(lambda x: x.std() / x.mean()).rename(columns= {'sum_quantity': 'CV'})

In [41]:
# добавляем столбец с классификацией товара и цветовым заполнением, где 
# X: Если CV ≤ 0,1 (очень стабильный спрос).
# Y: Если 0,1 < CV ≤ 0,25 (умеренная стабильность).
# Z: Если CV > 0,25 (нестабильный спрос).
df_cv['xyz'] = np.where(df_cv['CV'] <= 0.1, 'X', np.where(df_cv['CV'] <= 0.25, 'Y', 'Z'))
df_cv = df_cv.style.map(lambda x: 'background-color:green' if x == 'X' else 'background-color:yellow' if x == 'Y'\
else 'background-color:pink', subset=['xyz'])

In [42]:
df_cv

,product,CV,xyz
0,Camera,0.399434,Z
1,Charger,0.531511,Z
2,Drone,0.439487,Z
3,E-book Reader,0.571305,Z
4,External HDD,0.500797,Z
5,Fitness Tracker,0.348292,Z
6,Game Console,0.353174,Z
7,Graphics Card,0.252064,Z
8,Headphones,0.521949,Z
9,Keyboard,0.300573,Z


Большинство товаров (93%) попали в группу Z, что говорит о высокой нестабильности спроса. Это может быть вызвано:
Сезонностью
Рекламными кампаниями
Новизной продуктов
Малой регулярностью покупок
Только два товара — Laptop и Processor — показали умеренную стабильность (Y):
Эти позиции могут быть хорошими кандидатами для долгосрочного планирования закупок.
Они требуют менее агрессивного управления запасами.
Отсутствие X-группы может означать:
Период сбора данных был слишком коротким или нестабильным.
Вся продукция в целом имеет колеблящийся спрос.
Возможно, стоит пересмотреть критерии градации (например, снизить порог для X до CV ≤ 0.15).